<center>

## ***Company-based Capstone Project***
#### **Team ID		: C24-MR01**

Team Member	: 

(ML) M002D4KY2877 - Auvarifqi Putra Diandra

(ML) M010D4KY3370 - Rafi Madani

(ML) M002D4KY2625 - Iskandar Muda Rizky Parlambang

(MD) A010D4KY4202 - Muhammad Adryan Haska Putra

(MD) A297D4KX4551 - Vena Feranica

(CC) C002D4KY1032 - Muhammad Naufal

(CC) C459D4KY0090 - Jamaludin Ahmad Rifai



---

In [1]:
from typing import Dict, Text
from pathlib import Path

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [2]:
import pandas as pd

data_path = './data'
ratings_df = pd.read_csv(f"{data_path}/ratings.csv")
ratings_df.head(10)

,bucketized_user_age,movie_genres,movie_id,movie_title,raw_user_age,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,user_zip_code
0,45.0,7,357,One Flew Over the Cuckoo's Nest (1975),46.0,879024327,True,138,4,doctor,4.0,53211
1,25.0,4,709,Strictly Ballroom (1992),32.0,875654590,True,92,5,entertainment,2.0,80525
2,18.0,4,412,"Very Brady Sequel, A (1996)",24.0,882075110,True,301,17,student,4.0,55439
3,50.0,5,56,Pulp Fiction (1994),50.0,883326919,True,60,4,healthcare,4.0,06472
4,50.0,10,895,Scream 2 (1997),55.0,891409199,True,197,18,technician,3.0,75094
5,18.0,7,325,Crash (1996),19.0,876346551,False,601,1,artist,4.0,99687
6,18.0,2,95,Aladdin (1992),19.0,882064434,True,710,17,student,3.0,92020
7,25.0,0,92,True Romance (1993),34.0,875135363,True,833,21,writer,2.0,90019
8,25.0,4,425,Bob Roberts (1992),27.0,880844102,True,916,18,engineer,5.0,N2L5N
9,25.0,0,271,Starship Troopers (1997),32.0,884801053,True,940,2,administrator,2.0,02215


In [3]:
ratings_df.info()
total_rows, total_attributes = ratings_df.shape
print()
print('Jumlah data:', total_rows)
print("Jumlah atribut:", total_attributes)

ratings_df = ratings_df.drop_duplicates()

ratings_df.info()
total_rows, total_attributes = ratings_df.shape
print()
print('Jumlah data:', total_rows)
print("Jumlah atribut:", total_attributes)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   bucketized_user_age    100000 non-null  float64
 1   movie_genres           100000 non-null  int64  
 2   movie_id               100000 non-null  int64  
 3   movie_title            100000 non-null  object 
 4   raw_user_age           100000 non-null  float64
 5   timestamp              100000 non-null  int64  
 6   user_gender            100000 non-null  bool   
 7   user_id                100000 non-null  int64  
 8   user_occupation_label  100000 non-null  int64  
 9   user_occupation_text   100000 non-null  object 
 10  user_rating            100000 non-null  float64
 11  user_zip_code          100000 non-null  object 
dtypes: bool(1), float64(3), int64(5), object(3)
memory usage: 8.5+ MB

Jumlah data: 100000
Jumlah atribut: 12
<class 'pandas.core.frame.DataFrame'>

In [4]:
ratings_df.shape
np.random.seed(0)

In [5]:
ratings_df['like'] = np.random.randint(0, 2, size=len(ratings_df))

In [6]:
ratings_df.head()

,bucketized_user_age,movie_genres,movie_id,movie_title,raw_user_age,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,user_zip_code,like
0,45.0,7,357,One Flew Over the Cuckoo's Nest (1975),46.0,879024327,True,138,4,doctor,4.0,53211,0
1,25.0,4,709,Strictly Ballroom (1992),32.0,875654590,True,92,5,entertainment,2.0,80525,1
2,18.0,4,412,"Very Brady Sequel, A (1996)",24.0,882075110,True,301,17,student,4.0,55439,1
3,50.0,5,56,Pulp Fiction (1994),50.0,883326919,True,60,4,healthcare,4.0,06472,0
4,50.0,10,895,Scream 2 (1997),55.0,891409199,True,197,18,technician,3.0,75094,1


In [7]:
user_ids = ratings_df["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
movie_ids = ratings_df["movie_id"].unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}
ratings_df["user"] = ratings_df["user_id"].map(user2user_encoded)
ratings_df["movie"] = ratings_df["movie_id"].map(movie2movie_encoded)

num_users = len(user2user_encoded)
num_movies = len(movie_encoded2movie)
ratings_df["user_rating"] = ratings_df["user_rating"].values.astype(np.float32)
ratings_df["like"] = ratings_df["like"].values.astype(np.float32)
# min and max ratings will be used to normalize the ratings later
min_rating = min(ratings_df["user_rating"])
max_rating = max(ratings_df["user_rating"])
min_age = min(ratings_df["bucketized_user_age"])
max_age = max(ratings_df["bucketized_user_age"])

print(
    "Number of users: {}, Number of Movies: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_movies, min_rating, max_rating
    )
)

Number of users: 943, Number of Movies: 1682, Min rating: 1.0, Max rating: 5.0


In [8]:
df = ratings_df.sample(frac=1, random_state=42)
x = df[["user", "movie"]].values

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

df["rating_norm"] = df["user_rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating))
df["target"] = df.apply(lambda row: sigmoid(row["rating_norm"] + row["like"]), axis=1)
y = df["target"]

# Assuming training on 90% of the data and validating on 10%.
train_indices = int(0.8 * df.shape[0])
x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:],
)

print(x, y)
print("Jumlah data train: ", len(x_train))
print("Jumlah data val: ", len(x_val))

[[904 998]
 [485 647]
 [134 388]
 ...
 [394 162]
 [176 508]
 [171 477]] 75721    0.851953
80184    0.851953
19864    0.777300
76699    0.731059
92991    0.817574
           ...   
6265     0.880797
54886    0.817574
76820    0.817574
860      0.679179
15795    0.562177
Name: target, Length: 100000, dtype: float64
Jumlah data train:  80000
Jumlah data val:  20000


In [14]:
from tensorflow import keras
from keras import layers
from keras.utils import register_keras_serializable
from tensorflow.keras.utils import get_custom_objects

EMBEDDING_SIZE = 50

@register_keras_serializable()
class RecommenderNet(keras.Model):
    def __init__(self, num_users, num_movies, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_movies = num_movies
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.movie_embedding = layers.Embedding(
            num_movies,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.movie_bias = layers.Embedding(num_movies, 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        movie_vector = self.movie_embedding(inputs[:, 1])
        movie_bias = self.movie_bias(inputs[:, 1])
        dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
        x = dot_user_movie + user_bias + movie_bias
        return tf.nn.sigmoid(x)

    def get_config(self):
        config = super(RecommenderNet, self).get_config()
        config.update({
            'num_users': self.num_users,
            'num_movies': self.num_movies,
            'embedding_size': self.embedding_size
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

get_custom_objects().update({'RecommenderNet': RecommenderNet})

model = RecommenderNet(num_users, num_movies, EMBEDDING_SIZE)
model.build(input_shape=(None, 2))
model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    # metrics=[tf.keras.metrics.RootMeanSquaredError()]
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

model.summary()

print(f'Num users: {num_users}, Num movies: {num_movies}')
print(f'Embedding size: {EMBEDDING_SIZE}')
print(model.user_embedding)
print(model.movie_embedding)
print(model.user_bias)
print(model.movie_bias)

C:\Users\Iskandar\Documents\GitHub\nbs-ml\venv-collab\Lib\site-packages\keras\src\layers\layer.py:361: UserWarning: `build()` was called on layer 'recommender_net_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "recommender_net_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_5 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_6 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_7 (Embedding)              │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Num users: 943, Num movies: 1682
Embedding size: 50
<Embedding name=embedding_4, built=False>
<Embedding name=embedding_6, built=False>
<Embedding name=embedding_5, built=False>
<Embedding name=embedding_7, built=False>


In [15]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=1024,
    epochs=250,
    verbose=1,
    validation_data=(x_val, y_val),
)

Epoch 1/250
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6978 - root_mean_squared_error: 0.2669 - val_loss: 0.5831 - val_root_mean_squared_error: 0.1253
Epoch 2/250
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.9111 - root_mean_squared_error: 0.2433 - val_loss: 0.7030 - val_root_mean_squared_error: 0.2729
Epoch 3/250
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.0223 - root_mean_squared_error: 0.4593 - val_loss: 0.7719 - val_root_mean_squared_error: 0.3283
Epoch 4/250
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1472 - root_mean_squared_error: 0.2670 - val_loss: 0.8178 - val_root_mean_squared_error: 0.3604
Epoch 5/250
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6198 - root_mean_squared_error: 0.1569 - val_loss: 0.8000 - val_root_mean_squared_error: 0.3485
Epoch 6/250
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5944 - root_mean_squared_error: 0.1473 - val_loss: 0.7964 - val_root_mean_squared_error: 0.3461
Epoch 7/250
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss:

In [16]:
# Mendapatkan data frame dengan movieId unik
movie_df = ratings_df[['movie_id']].drop_duplicates()

# Memilih satu user secara acak
user_id = 1
movies_watched_by_user = df[df.user_id == user_id]
movies_not_watched = movie_df[
    ~movie_df["movie_id"].isin(movies_watched_by_user.movie_id.values)
]["movie_id"]
movies_not_watched = list(
    set(movies_not_watched).intersection(set(movie2movie_encoded.keys()))
)
movies_not_watched = [[movie2movie_encoded.get(x)] for x in movies_not_watched]
user_encoder = user2user_encoded.get(user_id)
user_movie_array = np.hstack(
    ([[user_encoder]] * len(movies_not_watched), movies_not_watched)
)

ratings = model.predict(user_movie_array).flatten()
top_ratings_indices = ratings.argsort()[-10:][::-1]
recommended_movie_ids = [
    movie_encoded2movie.get(movies_not_watched[x][0]) for x in top_ratings_indices
]

print("Showing recommendations for user: {}".format(user_id))
print("====" * 9)
print("Movies with high ratings from user")
print("----" * 8)
top_movies_user = (
    movies_watched_by_user.sort_values(by="user_rating", ascending=False)
    .head(5)
    .movie_id.values
)
movie_df_rows = movie_df[movie_df["movie_id"].isin(top_movies_user)]
for row in movie_df_rows.itertuples():
    print(row.movie_id, ":", ratings_df[ratings_df["movie_id"] == row.movie_id]["movie_title"].values[0])  # Asumsikan ada kolom 'genres' di movie_df

print("----" * 8)
print("Top 10 movie recommendations")
print("----" * 8)
recommended_movies = movie_df[movie_df["movie_id"].isin(recommended_movie_ids)]
for row in recommended_movies.itertuples():
    print(row.movie_id, ":", ratings_df[ratings_df["movie_id"] == row.movie_id]["movie_title"].values[0])  # Asumsikan ada kolom 'genres' di movie_df

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Showing recommendations for user: 1
Movies with high ratings from user
--------------------------------
19 : Antonia's Line (1995)
208 : Young Frankenstein (1974)
137 : Big Night (1996)
16 : French Twist (Gazon maudit) (1995)
198 : Nikita (La Femme Nikita) (1990)
--------------------------------
Top 10 movie recommendations
--------------------------------
603 : Rear Window (1954)
1169 : Fresh (1994)
1629 : Nico Icon (1995)
1599 : Someone Else's America (1995)
1536 : Aiqing wansui (1994)
626 : So Dear to My Heart (1949)
1443 : 8 Seconds (1994)
1616 : Desert Winds (1995)
1653 : Entertaining Angels: The Dorothy Day Story (1996)
1593 : Death in Brunswick (1991)


In [14]:
import os
import requests
import pickle

folder = './model'
if not os.path.exists(folder):
  os.makedirs(folder)

In [15]:
pickle.dump(df, open(folder + '/df.pkl', 'wb'))
pickle.dump(user2user_encoded, open(folder + '/user_id_encoded.pkl', 'wb'))
pickle.dump(movie2movie_encoded, open(folder + '/movie_id_encoded.pkl', 'wb'))

In [16]:
pickle.load(open(folder + '/df.pkl', 'rb'))

,bucketized_user_age,movie_genres,movie_id,movie_title,raw_user_age,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,user_zip_code,like,user,movie,rating_norm,target
75721,18.0,0,876,Money Talks (1997),19.0,874815542,True,817,17,student,4.0,60152,1.0,904,998,0.75,0.851953
80184,35.0,4,580,"Englishman Who Went Up a Hill, But Came Down a...",40.0,883869630,True,83,11,other,4.0,44133,1.0,485,647,0.75,0.851953
19864,35.0,4,393,Mrs. Doubtfire (1993),44.0,880088717,False,389,21,writer,2.0,83702,1.0,134,388,0.25,0.777300
76699,25.0,0,54,Outbreak (1995),30.0,888556814,True,774,17,student,1.0,80027,1.0,392,580,0.00,0.731059
92991,18.0,2,420,Alice in Wonderland (1951),22.0,874957140,False,712,17,student,3.0,54901,1.0,341,170,0.50,0.817574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,35.0,7,125,Phenomenon (1996),39.0,885852817,True,25,18,engineer,5.0,55107,1.0,141,39,1.00,0.880797
54886,50.0,5,156,Reservoir Dogs (1992),51.0,891397819,False,704,10,librarian,3.0,91105,1.0,520,675,0.50,0.817574
76820,25.0,10,288,Scream (1996),26.0,875129640,True,422,5,entertainment,3.0,94533,1.0,394,162,0.50,0.817574
860,18.0,4,168,Monty Python and the Holy Grail (1974),21.0,884207654,False,198,17,student,4.0,55414,0.0,176,508,0.75,0.679179


In [16]:
model.save(folder + '/model_collab.keras')

In [17]:
df.head()

,bucketized_user_age,movie_genres,movie_id,movie_title,raw_user_age,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,user_zip_code,like,user,movie,rating_norm,target
75721,18.0,0,876,Money Talks (1997),19.0,874815542,True,817,17,student,4.0,60152,1.0,904,998,0.75,0.851953
80184,35.0,4,580,"Englishman Who Went Up a Hill, But Came Down a...",40.0,883869630,True,83,11,other,4.0,44133,1.0,485,647,0.75,0.851953
19864,35.0,4,393,Mrs. Doubtfire (1993),44.0,880088717,False,389,21,writer,2.0,83702,1.0,134,388,0.25,0.777300
76699,25.0,0,54,Outbreak (1995),30.0,888556814,True,774,17,student,1.0,80027,1.0,392,580,0.00,0.731059
92991,18.0,2,420,Alice in Wonderland (1951),22.0,874957140,False,712,17,student,3.0,54901,1.0,341,170,0.50,0.817574


In [9]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [40]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

In [48]:
from tensorflow import keras
from keras import layers
from keras.utils import register_keras_serializable

EMBEDDING_SIZE = 50

class RecommenderNet(keras.Model):
    def __init__(self, num_users, num_movies, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_movies = num_movies
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.movie_embedding = layers.Embedding(
            num_movies,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.movie_bias = layers.Embedding(num_movies, 1)
        self.dense1 = layers.Dense(32, activation='relu')
        self.dense2 = layers.Dense(64, activation='relu')
        self.dense3 = layers.Dense(128, activation='relu')
        self.output_layer = layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        movie_vector = self.movie_embedding(inputs[:, 1])
        movie_bias = self.movie_bias(inputs[:, 1])
        dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
        # Add all the components (including bias)
        x = dot_user_movie + user_bias + movie_bias
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.output_layer(x)
        # rating_output = tf.nn.sigmoid(x)
        # like_output = tf.nn.sigmoid(x)
        # The sigmoid activation forces the rating to between 0 and 1
        return x


model = RecommenderNet(num_users, num_movies, EMBEDDING_SIZE)
model.build(input_shape=(None, 2))
model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    # metrics=[tf.keras.metrics.RootMeanSquaredError()]
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

model.summary()

print(f'Num users: {num_users}, Num movies: {num_movies}')
print(f'Embedding size: {EMBEDDING_SIZE}')
print(model.user_embedding)
print(model.movie_embedding)
print(model.user_bias)
print(model.movie_bias)

C:\Users\Iskandar\Documents\GitHub\nbs-ml\venv-collab\Lib\site-packages\keras\src\layers\layer.py:361: UserWarning: `build()` was called on layer 'recommender_net_11', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "recommender_net_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_48 (Embedding)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_49 (Embedding)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_50 (Embedding)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_51 (Embedding)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Num users: 943, Num movies: 1682
Embedding size: 50
<Embedding name=embedding_48, built=False>
<Embedding name=embedding_50, built=False>
<Embedding name=embedding_49, built=False>
<Embedding name=embedding_51, built=False>


In [49]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=64,
    epochs=50,
    verbose=1,
    validation_data=(x_val, y_val),
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.5818 - root_mean_squared_error: 0.1242 - val_loss: 0.5711 - val_root_mean_squared_error: 0.1051
Epoch 2/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5710 - root_mean_squared_error: 0.1049 - val_loss: 0.5708 - val_root_mean_squared_error: 0.1045
Epoch 3/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5705 - root_mean_squared_error: 0.1048 - val_loss: 0.5706 - val_root_mean_squared_error: 0.1040
Epoch 4/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5711 - root_mean_squared_error: 0.1044 - val_loss: 0.5706 - val_root_mean_squared_error: 0.1039
Epoch 5/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5698 - root_mean_squared_error: 0.1040 - val_loss: 0.5705 - val_root_mean_squared_error: 0.1036
Epoch 6/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5700 - root_mean_squared_error: 0.1032 - val_loss: 0.5705 - val_root_mean_squared_error: 0.1035
Epoch 7/50
1250/1250 ━━━━━━━━━━━━━━━━━━━

In [32]:
# Mendapatkan data frame dengan movieId unik
movie_df = ratings_df[['movie_id']].drop_duplicates()

# Memilih satu user secara acak
user_id = df.user_id.sample(1).iloc[0]
movies_watched_by_user = df[df.user_id == user_id]
movies_not_watched = movie_df[
    ~movie_df["movie_id"].isin(movies_watched_by_user.movie_id.values)
]["movie_id"]
movies_not_watched = list(
    set(movies_not_watched).intersection(set(movie2movie_encoded.keys()))
)
movies_not_watched = [[movie2movie_encoded.get(x)] for x in movies_not_watched]
user_encoder = user2user_encoded.get(user_id)
user_movie_array = np.hstack(
    ([[user_encoder]] * len(movies_not_watched), movies_not_watched)
)

ratings = model.predict(user_movie_array).flatten()
top_ratings_indices = ratings.argsort()[-10:][::-1]
recommended_movie_ids = [
    movie_encoded2movie.get(movies_not_watched[x][0]) for x in top_ratings_indices
]

print("Showing recommendations for user: {}".format(user_id))
print("====" * 9)
print("Movies with high ratings from user")
print("----" * 8)
top_movies_user = (
    movies_watched_by_user.sort_values(by="user_rating", ascending=False)
    .head(5)
    .movie_id.values
)
movie_df_rows = movie_df[movie_df["movie_id"].isin(top_movies_user)]
for row in movie_df_rows.itertuples():
    print(row.movie_id, ":", ratings_df[ratings_df["movie_id"] == row.movie_id]["movie_title"].values[0])  # Asumsikan ada kolom 'genres' di movie_df

print("----" * 8)
print("Top 10 movie recommendations")
print("----" * 8)
recommended_movies = movie_df[movie_df["movie_id"].isin(recommended_movie_ids)]
for row in recommended_movies.itertuples():
    print(row.movie_id, ":", ratings_df[ratings_df["movie_id"] == row.movie_id]["movie_title"].values[0])  # Asumsikan ada kolom 'genres' di movie_df

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Showing recommendations for user: 49
Movies with high ratings from user
--------------------------------
433 : Heathers (1989)
175 : Brazil (1985)
428 : Harold and Maude (1971)
209 : This Is Spinal Tap (1984)
47 : Ed Wood (1994)
--------------------------------
Top 10 movie recommendations
--------------------------------
265 : Hunt for Red October, The (1990)
285 : Secrets & Lies (1996)
269 : Full Monty, The (1997)
272 : Good Will Hunting (1997)
64 : Shawshank Redemption, The (1994)
251 : Shall We Dance? (1996)
318 : Schindler's List (1993)
316 : As Good As It Gets (1997)
943 : Killing Zoe (1994)
60 : Three Colors: Blue (1993)


In [128]:
# Konversi gender ke 0 dan 1
ratings_df["user_gender"] = ratings_df["user_gender"].astype(int)
ratings_df["bucketized_user_age"] = ratings_df["bucketized_user_age"].astype(int)


# Normalisasi usia bucket
min_age = min(ratings_df["bucketized_user_age"])
max_age = max(ratings_df["bucketized_user_age"])

# Shuffle data
df = ratings_df.sample(frac=1, random_state=42)
df["age_norm"] = df["bucketized_user_age"].apply(lambda x: (x - min_age) / (max_age - min_age))
x = df[["user", "movie", "user_gender"]].values
# x = df[["user", "movie"]].values

# Normalisasi rating dan target
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

df["rating_norm"] = df["user_rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating))
df["target"] = df.apply(lambda row: sigmoid(row["rating_norm"] + row["like"]), axis=1)
y = df["target"].values

# Split data
train_indices = int(0.8 * df.shape[0])
x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:],
)

In [129]:
class RecommenderNet(keras.Model):
    def __init__(self, num_users, num_movies, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_movies = num_movies
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.movie_embedding = layers.Embedding(
            num_movies,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.movie_bias = layers.Embedding(num_movies, 1)
        self.user_gender_embedding = layers.Embedding(
            2,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.user_gender_bias = layers.Embedding(2, 1)
        self.dense1 = layers.Dense(32, activation='relu')
        self.dense2 = layers.Dense(64, activation='relu')
        self.dense3 = layers.Dense(128, activation='relu')
        self.dropout = layers.Dropout(0.5)
        self.output_layer = layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        movie_vector = self.movie_embedding(inputs[:, 1])
        movie_bias = self.movie_bias(inputs[:, 1])
        user_gender_vector = self.user_gender_embedding(inputs[:, 2])
        user_gender_bias = self.user_gender_bias(inputs[:, 2])
        # dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
        # dot_user_movie = tf.tensordot(dot_user_movie, user_gender_vector, 2)
        # Add all the components (including bias)
        dot_user_movie = tf.reduce_sum(user_vector * movie_vector, axis=1, keepdims=True)
        dot_user_movie_gender = dot_user_movie + tf.reduce_sum(user_vector * user_gender_vector, axis=1, keepdims=True)
        dot_user_movie_gender = dot_user_movie_gender + tf.reduce_sum(movie_vector * user_gender_vector, axis=1, keepdims=True)
        x = dot_user_movie_gender + user_bias + movie_bias + user_gender_bias
        x = self.dense1(x)
        x = self.dropout(x)
        x = self.dense2(x)
        x = self.dropout(x)
        x = self.dense3(x)
        x = self.dropout(x)
        x = self.output_layer(x)
        # rating_output = tf.nn.sigmoid(x)
        # like_output = tf.nn.sigmoid(x)
        # The sigmoid activation forces the rating to between 0 and 1
        return x

EMBEDDING_SIZE = 50
model = RecommenderNet(num_users, num_movies, EMBEDDING_SIZE)
model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

model.build(input_shape=(None, 3))
model.summary()


C:\Users\Iskandar\Documents\GitHub\nbs-ml\venv-collab\Lib\site-packages\keras\src\layers\layer.py:361: UserWarning: `build()` was called on layer 'recommender_net_35', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "recommender_net_35"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_184 (Embedding)            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_185 (Embedding)            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_186 (Embedding)            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_187 (Embedding)            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_188 (Embedding)            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_189 (Embedding)            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_123 (Dense)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_124 (Dense)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_125 (Dense)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_126 (Dense)                    │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [130]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=1024,
    epochs=250,
    verbose=1,
    validation_data=(x_val, y_val),
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/250
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.6200 - root_mean_squared_error: 0.1803 - val_loss: 0.5703 - val_root_mean_squared_error: 0.1035
Epoch 2/250
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5693 - root_mean_squared_error: 0.1015 - val_loss: 0.5701 - val_root_mean_squared_error: 0.1032
Epoch 3/250
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5691 - root_mean_squared_error: 0.1005 - val_loss: 0.5701 - val_root_mean_squared_error: 0.1032
Epoch 4/250
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5688 - root_mean_squared_error: 0.1004 - val_loss: 0.5703 - val_root_mean_squared_error: 0.1035
Epoch 5/250
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5689 - root_mean_squared_error: 0.1004 - val_loss: 0.5703 - val_root_mean_squared_error: 0.1035
Epoch 6/250
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5679 - root_mean_squared_error: 0.0997 - val_loss: 0.5702 - val_root_mean_squared_error: 0.1034
Epoch 7/250
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss:

In [131]:
# Mendapatkan data frame dengan movieId unik
movie_df = ratings_df[['movie_id']].drop_duplicates()

# Memilih satu user secara acak
user_id = df.user_id.sample(1).iloc[0]
movies_watched_by_user = df[df.user_id == user_id]
movies_not_watched = movie_df[
    ~movie_df["movie_id"].isin(movies_watched_by_user.movie_id.values)
]["movie_id"]
movies_not_watched = list(
    set(movies_not_watched).intersection(set(movie2movie_encoded.keys()))
)
movies_not_watched = [[movie2movie_encoded.get(x)] for x in movies_not_watched]
user_encoder = user2user_encoded.get(user_id)
user_gender = df[df["user"] == user_id]["user_gender"].iloc[0]
user_movie_array = np.hstack(
    ([[user_encoder]] * len(movies_not_watched), movies_not_watched, [[user_gender]] * len(movies_not_watched))
)

ratings = model.predict(user_movie_array).flatten()
top_ratings_indices = ratings.argsort()[-10:][::-1]
recommended_movie_ids = [
    movie_encoded2movie.get(movies_not_watched[x][0]) for x in top_ratings_indices
]

print("Showing recommendations for user: {}".format(user_id))
print("====" * 9)
print("Movies with high ratings from user")
print("----" * 8)
top_movies_user = (
    movies_watched_by_user.sort_values(by="user_rating", ascending=False)
    .head(5)
    .movie_id.values
)
movie_df_rows = movie_df[movie_df["movie_id"].isin(top_movies_user)]
for row in movie_df_rows.itertuples():
    print(row.movie_id, ":", ratings_df[ratings_df["movie_id"] == row.movie_id]["movie_title"].values[0])  # Asumsikan ada kolom 'genres' di movie_df

print("----" * 8)
print("Top 10 movie recommendations")
print("----" * 8)
recommended_movies = movie_df[movie_df["movie_id"].isin(recommended_movie_ids)]
for row in recommended_movies.itertuples():
    print(row.movie_id, ":", ratings_df[ratings_df["movie_id"] == row.movie_id]["movie_title"].values[0])  # Asumsikan ada kolom 'genres' di movie_df

52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Showing recommendations for user: 165
Movies with high ratings from user
--------------------------------
258 : Contact (1997)
326 : G.I. Jane (1997)
318 : Schindler's List (1993)
169 : Wrong Trousers, The (1993)
651 : Glory (1989)
--------------------------------
Top 10 movie recommendations
--------------------------------
285 : Secrets & Lies (1996)
302 : L.A. Confidential (1997)
641 : Paths of Glory (1957)
483 : Casablanca (1942)
817 : Frisk (1995)
60 : Three Colors: Blue (1993)
1113 : Mrs. Parker and the Vicious Circle (1994)
513 : Third Man, The (1949)
1459 : Madame Butterfly (1995)
1443 : 8 Seconds (1994)


In [135]:
# Konversi gender ke 0 dan 1
ratings_df["user_gender"] = ratings_df["user_gender"].astype(int)
ratings_df["bucketized_user_age"] = ratings_df["bucketized_user_age"].astype(int)


# Normalisasi usia bucket
min_age = min(ratings_df["bucketized_user_age"])
max_age = max(ratings_df["bucketized_user_age"])

# Shuffle data
df = ratings_df.sample(frac=1, random_state=42)
df["age_norm"] = df["bucketized_user_age"].apply(lambda x: (x - min_age) / (max_age - min_age))
x = df[["user", "movie", "user_gender", "age_norm"]].values
# x = df[["user", "movie"]].values

num_ages = len(df["age_norm"].unique())

# Normalisasi rating dan target
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

df["rating_norm"] = df["user_rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating))
df["target"] = df.apply(lambda row: sigmoid(row["rating_norm"] + row["like"]), axis=1)
y = df["target"].values

# Split data
train_indices = int(0.8 * df.shape[0])
x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:],
)

In [141]:
class RecommenderNet(keras.Model):
    def __init__(self, num_users, num_movies, num_ages, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_movies = num_movies
        self.num_ages = num_ages
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.movie_embedding = layers.Embedding(
            num_movies,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.movie_bias = layers.Embedding(num_movies, 1)
        self.user_gender_embedding = layers.Embedding(
            2,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.user_gender_bias = layers.Embedding(2, 1)
        self.age_embedding = layers.Embedding(
            num_ages,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.age_bias = layers.Embedding(num_ages, 1)
        self.dense1 = layers.Dense(32, activation='relu')
        self.dense2 = layers.Dense(64, activation='relu')
        self.dense3 = layers.Dense(128, activation='relu')
        self.dropout = layers.Dropout(0.5)
        self.output_layer = layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        movie_vector = self.movie_embedding(inputs[:, 1])
        movie_bias = self.movie_bias(inputs[:, 1])
        user_gender_vector = self.user_gender_embedding(inputs[:, 2])
        user_gender_bias = self.user_gender_bias(inputs[:, 2])
        age_vector = self.age_embedding(inputs[:, 3])
        age_bias = self.age_bias(inputs[:, 3])
        # dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
        # dot_user_movie = tf.tensordot(dot_user_movie, user_gender_vector, 2)
        # Add all the components (including bias)
        dot_user_movie = tf.reduce_sum(user_vector * movie_vector, axis=1, keepdims=True)
        dot_user_movie_gender = dot_user_movie + tf.reduce_sum(user_vector * user_gender_vector, axis=1, keepdims=True)
        dot_user_movie_gender = dot_user_movie_gender + tf.reduce_sum(movie_vector * user_gender_vector, axis=1, keepdims=True)
        dot_user_movie_gender_age = dot_user_movie_gender + tf.reduce_sum(movie_vector * age_vector, axis=1, keepdims=True)
        dot_user_movie_gender_age = dot_user_movie_gender_age + tf.reduce_sum(user_vector * age_vector, axis=1, keepdims=True)
        dot_user_movie_gender_age = dot_user_movie_gender_age + tf.reduce_sum(user_gender_vector * age_vector, axis=1, keepdims=True)
        x = dot_user_movie_gender_age + user_bias + movie_bias + user_gender_bias +age_bias
        x = self.dense1(x)
        x = self.dropout(x)
        x = self.dense2(x)
        x = self.dropout(x)
        x = self.dense3(x)
        x = self.dropout(x)
        x = self.output_layer(x)
        # rating_output = tf.nn.sigmoid(x)
        # like_output = tf.nn.sigmoid(x)
        # The sigmoid activation forces the rating to between 0 and 1
        return x

EMBEDDING_SIZE = 50
model = RecommenderNet(num_users, num_movies, num_ages, EMBEDDING_SIZE)
model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

model.build(input_shape=(None, 4))
model.summary()


C:\Users\Iskandar\Documents\GitHub\nbs-ml\venv-collab\Lib\site-packages\keras\src\layers\layer.py:361: UserWarning: `build()` was called on layer 'recommender_net_38', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "recommender_net_38"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_206 (Embedding)            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_207 (Embedding)            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_208 (Embedding)            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_209 (Embedding)            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_210 (Embedding)            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_211 (Embedding)            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_212 (Embedding)            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_213 (Embedding)            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_135 (Dense)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_136 (Dense)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_137 (Dense)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_18 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_138 (Dense)                    │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [142]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=1024,
    epochs=250,
    verbose=1,
    validation_data=(x_val, y_val),
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/250
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.6041 - root_mean_squared_error: 0.1556 - val_loss: 0.5712 - val_root_mean_squared_error: 0.1042
Epoch 2/250
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5700 - root_mean_squared_error: 0.1022 - val_loss: 0.5704 - val_root_mean_squared_error: 0.1035
Epoch 3/250
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5684 - root_mean_squared_error: 0.1005 - val_loss: 0.5703 - val_root_mean_squared_error: 0.1033
Epoch 4/250
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5686 - root_mean_squared_error: 0.1002 - val_loss: 0.5704 - val_root_mean_squared_error: 0.1035
Epoch 5/250
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5679 - root_mean_squared_error: 0.0996 - val_loss: 0.5704 - val_root_mean_squared_error: 0.1036
Epoch 6/250
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5682 - root_mean_squared_error: 0.0997 - val_loss: 0.5705 - val_root_mean_squared_error: 0.1038
Epoch 7/250
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss:

In [148]:
# Mendapatkan data frame dengan movieId unik
movie_df = ratings_df[['movie_id']].drop_duplicates()

# Memilih satu user secara acak
user_id = df.user_id.sample(1).iloc[0]
movies_watched_by_user = df[df.user_id == user_id]
movies_not_watched = movie_df[
    ~movie_df["movie_id"].isin(movies_watched_by_user.movie_id.values)
]["movie_id"]
movies_not_watched = list(
    set(movies_not_watched).intersection(set(movie2movie_encoded.keys()))
)
movies_not_watched = [[movie2movie_encoded.get(x)] for x in movies_not_watched]
user_encoder = user2user_encoded.get(user_id)
user_gender = df[df["user"] == user_id]["user_gender"].iloc[0]
user_age = df[df["user"] == user_id]["age_norm"].iloc[0]
user_movie_array = np.hstack(
    ([[user_encoder]] * len(movies_not_watched), movies_not_watched, [[user_gender]] * len(movies_not_watched), [[user_age]] * len(movies_not_watched))
)

ratings = model.predict(user_movie_array).flatten()
top_ratings_indices = ratings.argsort()[-10:][::-1]
recommended_movie_ids = [
    movie_encoded2movie.get(movies_not_watched[x][0]) for x in top_ratings_indices
]

print("Showing recommendations for user: {}".format(user_id))
print("====" * 9)
print("Movies with high ratings from user")
print("----" * 8)
top_movies_user = (
    movies_watched_by_user.sort_values(by="user_rating", ascending=False)
    .head(5)
    .movie_id.values
)
movie_df_rows = movie_df[movie_df["movie_id"].isin(top_movies_user)]
for row in movie_df_rows.itertuples():
    print(row.movie_id, ":", ratings_df[ratings_df["movie_id"] == row.movie_id]["movie_title"].values[0])  # Asumsikan ada kolom 'genres' di movie_df

print("----" * 8)
print("Top 10 movie recommendations")
print("----" * 8)
recommended_movies = movie_df[movie_df["movie_id"].isin(recommended_movie_ids)]
for row in recommended_movies.itertuples():
    print(row.movie_id, ":", ratings_df[ratings_df["movie_id"] == row.movie_id]["movie_title"].values[0])  # Asumsikan ada kolom 'genres' di movie_df

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 823us/step
Showing recommendations for user: 216
Movies with high ratings from user
--------------------------------
56 : Pulp Fiction (1994)
7 : Twelve Monkeys (1995)
22 : Braveheart (1995)
237 : Jerry Maguire (1996)
238 : Raising Arizona (1987)
--------------------------------
Top 10 movie recommendations
--------------------------------
127 : Godfather, The (1972)
603 : Rear Window (1954)
344 : Apostle, The (1997)
515 : Boot, Das (1981)
483 : Casablanca (1942)
474 : Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)
647 : Ran (1985)
513 : Third Man, The (1949)
1459 : Madame Butterfly (1995)
18 : White Balloon, The (1995)
